# Title: Evaluation and Visualization
# Description: Evaluates trained model and visualizes metrics for thesis figures.

In [ ]:
import json
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import torch
import pandas as pd
from src.evaluate import evaluate_model
from src.model_architecture import FusionModel
from src.utils import load_checkpoint
from src.data_loading import make_dataloaders
from src import config

# Paths
test_csv = "data/processed/rsna_processed.csv"
checkpoint_path = "experiments/checkpoints/epoch10_valauc0.7290.pt"  # example

# Load test dataframe
df_test = pd.read_csv(test_csv)

# Prepare DataLoader
_, _, test_loader = make_dataloaders(df_test, df_test, df_test, batch_size=16)

# Load model and checkpoint
model = FusionModel(metadata_dim=len(config.METADATA_FEATURES))
ckpt = load_checkpoint(checkpoint_path, model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Evaluate


In [ ]:
results = evaluate_model(model, test_loader, device=device)
metrics = results["metrics"]
print("Evaluation metrics:", json.dumps(metrics, indent=2))

# Plot ROC curve


In [ ]:
y_true, y_prob = results["y_true"], results["y_prob"]
fpr, tpr, _ = roc_curve(y_true, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - RSNA Test Set")
plt.legend()
plt.grid(True)
plt.show()

# Confusion Matrix

In [ ]:
y_pred = (y_prob >= 0.5).astype(int)
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(cm)
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix - RSNA Test Set")
plt.show()